#0. Installing Transformers and Importing Dependencies


In [2]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 7.3 MB/s 
     |████████████████████████████████| 163 kB 63.7 MB/s 
     |████████████████████████████████| 7.6 MB 49.9 MB/s 


In [3]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

#1. Load Summarization Pipeline





In [4]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

#2. Get Blog Post from Medium


In [5]:
URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
URL = "https://www.cricbuzz.com/cricket-news/124217/the-dk-project-building-a-modern-day-t20-hyperspecialist"

In [6]:
r = requests.get(URL)


In [7]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [8]:
ARTICLE


" {{suggest.tag}} Search for “” The DK Project: Building a modern-day T20 hyperspecialist  It's 2022. Dinesh Karthik is 37. It's a day out from India's first official T20 World Cup warm-up. He walks into the nets at the Gabba in Brisbane and to the second ball he faces, he shuffles across, squats and then sets up to reverse-lap the ball over the imaginary wicket-keeper. He bats for 20 minutes in a session of ferrety movements, improvisations and premeditations. At the end of it, he has a word with the batting coach and then pauses to get a drink...   It's more than four years since Karthik played the last of his 26 Test matches; more than three since his ODI career stalled with India being knocked out of the World Cup. In 2021, at the age of 36, Karthik joined Sky Sports' commentary panel for the English summer, and most observers assumed his transition from international cricketer to media pundit was complete.   But behind the snazzy suits, the tinted glasses and his commentary quips,

#3. Chunk Text


In [9]:
max_chunk = 500


In [10]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [11]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [12]:
len(chunks)

13

#4. Summarize Text


In [13]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [14]:
res[0]

{'summary_text': ' Dinesh Karthik and Abhishek Nayar are building a modern-day T20 hyperspecialist . The DK Project is the brainchild of former cricketer and long-time coach and confidant Nayar . The pair have been going from remote grounds to another with an expensive training regime .'}

In [15]:
' '.join([summ['summary_text'] for summ in res])

" Dinesh Karthik and Abhishek Nayar are building a modern-day T20 hyperspecialist . The DK Project is the brainchild of former cricketer and long-time coach and confidant Nayar . The pair have been going from remote grounds to another with an expensive training regime .  DK: I like to go to different places and practice on different soils, different bowlers, different conditions. I just feel it's more challenging and I think it comes more from where he is going to be playing the next tournament and we try and go to similar conditions . DK: If the size of the ground is slightly big, I enjoy it a lot more, because if I learn to hit on bigger grounds, you know that feeling that you get is very good .  Was the decision to mould Karthik completely as a finisher a conscious decision or circumstance driven? DK: It was only around 2018-19 that I made a conscious effort to bat slightly lower down the order for KKR . I realized, with me playing finisher, the pressure of the game was different . 

In [16]:
text = ' '.join([summ['summary_text'] for summ in res])

#5. Output to Text File


In [17]:
with open('blogsummary.txt', 'w') as f:
    f.write(text)